In [37]:
import os
import sys
#os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,50).__str__()
import cv2 as cv
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

In [60]:
def identify_alum(filename):
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)

    # convert to hsv and get saturation channel
    sat = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:,:,1]

    # do a little Gaussian filtering
    blur = cv2.GaussianBlur(sat, (3,3), 0)
    #(3,3) is the Gaussian kernel, 
    # blurring the image mitigates noise

    # threshold and invert to create initial mask
    mask = 255 - cv2.threshold(blur, 15, 255, cv2.THRESH_BINARY)[1]
    # 100 is threshold val, 255 is maxval, THRESH_Bin is thresholding type
    # if >100 then set image's pixel to max, else set to 0

    # apply morphology close to fill interior regions in mask
    #kernel = np.ones((15,15), np.uint8)
    #mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    # remove morphology as for now not worry about filling regions
    
    '''
    # get outer contours from inverted mask and get the largest (presumably only one due to morphology filtering)
    #cntrs = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntrs = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # cv2 approx_simple will approximate only 4 points
    cntrs = cntrs[0] if len(cntrs) == 2 else cntrs[1]
    result = img.copy()
    area_thresh = 0
    for c in cntrs:
        area = cv2.contourArea(c)
        if area > area_thresh:
            area = area_thresh
            big_contour = c

    # draw largest contour
    cv2.drawContours(result, [big_contour], -1, (0,0,255), 2)
    '''
    result = img.copy()
    contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnt = contours[4]
    cv.drawContours(result, [cnt], 0, (0,255,0), 3)

    # write result to disk
    #cv2.imwrite("shiny_mask.png", mask)
    #cv2.imwrite("shiny_outline.png", result)

    # display it
    #cv2.imshow("IMAGE", img)
    #cv2.imshow("MASK", mask)
    #cv2.imshow("RESULT", result)
    cv2.waitKey(0)
    plt.imshow(img)
    plt.show()
    plt.imshow(mask)
    plt.show()
    plt.imshow(result)
    plt.show()

In [79]:
def identify_alum2(filename):
    im = cv.imread(filename)
    imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
    #sat = cv.cvtColor(im, cv.COLOR_RGB2HSV)[:,:,1]
    blur = cv.GaussianBlur(imgray, (5,5), 0)
    #ret, thresh = cv.threshold(imgray, thresh=170, maxval=255, type=0)
    ret, thresh = cv.threshold(blur, thresh=170, maxval=255, type=cv.THRESH_BINARY)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    #cv.drawContours(im, contours, -1, (0,255,0), 3)
    
    rects = []
    areas = []
    for cnt in contours:
            rect = cv.boundingRect(cnt)
            rects.append(rect)
            areas.append(rect[2]*rect[3])
    i_max_area = areas.index(max(areas))
    print(rects[i_max_area])
    box = cv.boxPoints(rects[i_max_area])
    box = np.int0(box)
    cv.drawContours(im, [box], -1, (0,255,0), 3)
    plt.imshow(im)
    plt.show()
    plt.imshow(thresh)
    plt.show()

In [80]:
#identify_alum('Straight_On.png')
identify_alum2('SmallPieceWithFlash.jpeg')

(367, 280, 307, 983)


error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'boxPoints'
> Overload resolution failed:
>  - Can't parse 'box' as RotatedRect. Expected sequence length 3, got 4
>  - Can't parse 'box' as RotatedRect. Expected sequence length 3, got 4
